# Churn Prediction

**"Churn rate"** , or simply **"churn"**, represents the rate of customer base evasion. In services such as Spotify or Netflix, it would represent the rate of subscription cancellations.



It is of utmost importance for management, and its analysis over time can show that there is a problem that needs to be addressed.

"Churn" can also be used to identify potential cancellations in advance and promote targeted actions to try to retain such customers. This metric should receive attention because the Cost of Customer Acquisition (CAC) is typically higher than the cost to retain them. In other words, a high value for the churn rate is undesirable.

## Data Acquisition

The data used in this project was originally made available at [IBM Developer](https://developer.ibm.com/technologies/data-science/patterns/predict-customer-churn-using-watson-studio-and-jupyter-notebooks/#), and deals with a typical problem of a telecommunications company. The complete dataset can be found [here](https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv).

Although there is no explicit information available, the column names allow for an understanding of the problem

In [7]:
# installing the necessary packages
!pip install scikit-learn -q
!pip install --upgrade imbalanced-learn

In [8]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# importing the necessary packages
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# importing the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


# random seed
np.random.seed(42)

# importing the data
DATA_PATH = "https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(DATA_PATH)

# checking the first entries
df.head()

ModuleNotFoundError: No module named 'lightgbm'

Visualizing the first 5 entries allows us to understand structure of the dataset. Even though there is no extra material with more details about how the data was collected and organized, it is stills possible to understand and work with this data.

As shown below, the dataset is composed of 7043 entries and 21 variables.

In [ ]:
print("Rows:\t\t{}".format(df.shape[0]))
print("Columns:\t{}".format(df.shape[1]))

## Exploratory Data Analysis

To start off this project, let's check the integrity and usability of the dataset.

To do this, I will print out the amount of missing data, the column names, the variable types per column, and the unique values per column. This last one, specifically, will help in identifying potentially categorical variables.

In [ ]:
print("Missing Data (total):\t{}\n".format(df.isnull().sum().values.sum()))
print(df.isnull().sum() / df.shape[0])
print("\nFeatures:\n{}\n".format(df.columns.values))
print("Unique Values (by column):\n{}\n".format(df.nunique()))
print("Variable Type (by column)\n{}".format(df.dtypes))

Let's go over the highlights:


* `Churn` is the target variable, indicating the situation of the client regarding this metric.
* `TotalCharges` should be a `float`, but is currently `string`.
    * As of right now, there aren't any missing values. However, we have string values that could be understood as missing.
* Several features must be converted to numeric.
    * There are several binary features, including the target variable, as well as categorical values, each of them indicating a different category.


## Data Cleaning

When analyzing `TotalCharges`, we have identified that the missing data was entered as the string `""`. If we simply try to transform it into float(""), the interpreter returns an error.

So, in order to convert it properly, I created a function to handle this exception. In the end, I identified missing data and replaced it with the median value of the column.

Below, I also took the opportunity to delete the customerID column, which would be unnecessary throughout this project.

In [ ]:
def converter_str_float(entry):
    """
    Converting string objects into float 

    # Arguments
        entry: string

    # Returns
        If possible, value converted as float.
        NaN, if the transformation isn't possible.
    """

    try:
        return float(entry)
    except ValueError:
        return np.nan

# copying the dataframe
df_clean = df.copy()

# removing custumerID
df_clean.drop("customerID", axis=1, inplace=True)

# converting strings in TotalCharges to float
df_clean["TotalCharges"] = df_clean["TotalCharges"].apply(converter_str_float)

# checking for missing data
print("Missing Data in 'TotalCharges' (before replacing with Median):\t{}\n".format(df_clean["TotalCharges"].isnull().sum()))

# replacing NaN with Median
TotalChargesMedian = df_clean.TotalCharges.median()
df_clean["TotalCharges"].fillna(TotalChargesMedian, inplace=True)

Even though the dataset is mostly composed of binary or categorical variables, it is possible to see below, using the `describe` method, that there are no indications of the presence of outliers.

In [ ]:
df_clean.describe() 

Analyzing boxplots for `MonthlyCharges` and `TotalCharges`, the distribution seems to reinforce our initial hypothesis. 

There doesn't seem to be data points above or below the limits.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 7))
sns.boxplot(df_clean['MonthlyCharges'], ax=ax[0])
sns.boxplot(df_clean['TotalCharges'], ax=ax[1])
plt.tight_layout()

In this case, we can take the variable `tenure`, which indicates how long the client has been a client, as an indication of the quality of the relationship of the client with the company.

Since we don't have an official documentation regarding the dataset, let's assume the time unit used in this variable is months.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
sns.countplot(df['tenure'])
plt.tight_layout()

When dealing with categorical variables, it's important to check for unique values. It allows us to identify when different labels can mean the same thing.

For example, you can see below that we have the following entries:`No`, `No phone service` and `No internet service`, which can mean the same thing.

It's a valid point. However, since we can't know for sure, let's consider them as different values.

In [ ]:
# checking unique values for object variables
np.unique(df_clean.select_dtypes('object').values)

To identify if the dataset shows any kind of bias, let's check the count of `Male` and `Female`.

This type of bias doesn't seem relevant in this specific case. However, there are many instances where it could be a real problem. For instance, if we were working on a project related to a hiring process.

As we can see below, the distribution is balanced.

In [ ]:
df.head()

In [ ]:
sns.countplot(x='gender', data=df);

Last but not least, it's important to check whether the dataset is balanced or not, when it comes to the target variable.

As we can see below, `No` is a lot more common than `Yes`. This must be taken into account when building a **Machine Learning** model for this dataset.

In [ ]:
sns.countplot(x='Churn', data=df);

## Data Pre-Processing for Machine Learning

Let's prepare the data in order to fit into a Machine Learning model.

### The Process

For binary features, I will use `LabelEncoder`. This includes our target variable, `Churn`.

The next step is separating numerical and categorical variables. Categorical variables will be transformed into Dummy Variables, so they can be used by different models.

In [ ]:
# pre-processing variables
binary_var = df_clean.nunique()[df_clean.nunique() == 2].keys().tolist()
num_var = [col for col in df_clean.select_dtypes(['int', 'float']).columns.tolist() if col not in binary_var]
cat_var = [col for col in df_clean.columns.tolist() if col not in binary_var + num_var]

# creating a copy of the dataframe
df_proc = df_clean.copy()

# Label Encoding for Binary Variables
le = LabelEncoder()
for i in binary_var:
    df_proc[i] = le.fit_transform(df_proc[i])

# Encoding for multi-class variables
df_proc = pd.get_dummies(df_proc, columns=cat_var)

# checking the new dataframe
df_proc.head()

## Machine Learning

With the previous pre-processing, the data is ready to be used in a Machine Learning model.

Before we go into hyperparameter tuning, feature engineering, feature selection, etc, let's fit the data into a simple model, to be used as baseline for the project.

However, it's important to split the data here, before we move forward.

In [ ]:
# isolating the target variable
X = df_proc.drop('Churn', axis=1)
y = df_proc['Churn']

# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y)

I will use `cross-validation` to estimate the error for our baseline.

To make the process easier, I created a `val_model` function.

The models will be evaluated on `recall`.

In [ ]:
def val_model(X, y, clf, quite=False):
    """
    Cross Validation Function.

    # Arguments
        X: DataFrame, independent variables.
        y: Series, vector, target variable.
        clf: Scikit-learn Classifier.
        quiet: bool, indicating whether the function should print the results.

    # Returns
        float, cross-validation avergae score.
    """
    
    X = np.array(X)
    y = np.array(y)

    pipeline = make_pipeline(StandardScaler(), clf)
    scores = cross_val_score(pipeline, X, y, scoring='recall')

    if quite == False:
        print("Recall: {:.2f} (+/- {:.2f})".format(scores.mean(), scores.std()))
    
    return scores.mean()

For the baseline, I will use `Random Forest`, without any type of tuning, with the training data.

In [ ]:
# creating baseline
rf = RandomForestClassifier()
score_baseline = val_model(X_train, y_train, rf)

Now that we have a baseline, let's move on, and try to improve our results.

For this project, after some testing I've found that `Under Sampling` yielded the best results. Also, following recommendations for Under Sampling, I've decided to apply `Standard Scaler` to the data.

**Warning**

Keep in mind that only the **training** data will be balanced. The test data must remain untouched.

In [ ]:
# standardizing the data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)


# oversample = SMOTE()
# X_train, y_train = oversample.fit_sample(X_train, y_train)

# under sampling
rus = RandomUnderSampler()
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

Now, let's try fitting diferrent models for cross validation, to find out which model works best for our specific problem.

* Random Forest
* Decision Tree
* Stochastic Gradient Descent
* SVC
* Logistic Regression
* LightGBM



In [ ]:
# instantiating the models
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
sgdc = SGDClassifier()
svc = SVC()
lr = LogisticRegression()
xgb = XGBClassifier()
lgbm = LGBMClassifier()

model = []
recall = []
# evaluating results (recall)
for clf in (rf, dt, sgdc, svc, lr, xgb, lgbm):
    model.append(clf.__class__.__name__)
    recall.append(val_model(X_train_rus, y_train_rus, clf, quite=True))

pd.DataFrame(data=recall, index=model, columns=['Recall'])

Given the characteristics of the problem and the performance during previous executions, I opted for XGBoost, which was practically tied with Logistic Regression.

My understanding is that parameter tuning can further improve the quality of the solution. After some testing adjusting parameters for Logistic Regression and some of the other top performing models, XGBoost performed better after optimization.

## Hyperparameter Optimization

XGBoost has numerous parameters. Some have a greater impact on the model's quality, while others have less. A good practice is to set a learning rate and number of estimators, tune other parameters, and then check other learning rates.

Below, I will adjust the number of estimators. The model was instantiated with `learning_rate=0.1`.

In [ ]:
xgb = XGBClassifier(learning_rate=0.1)

param_grid = {
 'n_estimators':range(0,1000,50),
}

# identifying the best parameters
kfold = StratifiedKFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(xgb, param_grid, scoring="recall", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train_rus, y_train_rus)

# checking results
print("Best: {} for {}".format(grid_result.best_score_, grid_result.best_params_))

With the number of estimators set at 50, let's optimize `max_depth` and `min_child_weight`.

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=50)

param_grid = {
 'max_depth':range(1,8,1),
 'min_child_weight':range(1,5,1)
}

# identifying the best parameters
kfold = StratifiedKFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(xgb, param_grid, scoring="recall", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train_rus, y_train_rus)

# checking results
print("Best: {} for {}".format(grid_result.best_score_, grid_result.best_params_))

After optimization, we have found that the ideal parameters are `max_depth=1` and `min_child_weight=1`.

Next step, let''s optimize `gamma`.

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=50, max_depth=1, min_child_weight=1)

param_grid = {
 'gamma':[i/10.0 for i in range(0,5)]
}

# identifying the best parameters
kfold = StratifiedKFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(xgb, param_grid, scoring="recall", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train_rus, y_train_rus)

# checking results
print("Best: {} for {}".format(grid_result.best_score_, grid_result.best_params_))

Finally, with `gamma=0`, let's test other values for `learning_rate` to make sure we have the best parameters.

In [ ]:
xgb = XGBClassifier(n_estimators=50, max_depth=1, min_child_weight=1, gamma=0.0)

param_grid = {
 'learning_rate':[0.001, 0.01, 0.1, 1]
}

# identifying the best parameters
kfold = StratifiedKFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(xgb, param_grid, scoring="recall", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train_rus, y_train_rus)

# checking results
print("Best: {} for {}".format(grid_result.best_score_, grid_result.best_params_))

Assuming `recall` is the most important metric for this projetct, we can see that a lower value for `learning_rate` was the best call.

Now, it's time to train the model, and check the results on the test dataset. This is going to be the first time the model receives the test dataset.

In [ ]:
# final model
xgb = XGBClassifier(learning_rate=0.001 , n_estimators=50, max_depth=1, min_child_weight=1, gamma=0.0)
xgb.fit(X_train_rus, y_train_rus)

# making predictions
X_test = scaler.transform(X_test)
y_pred = xgb.predict(X_test)

# Classification Report
print(classification_report(y_test, y_pred))

# plotting area under the curve
print("AUC: {:.4f}\\n".format(roc_auc_score(y_test, y_pred)))

# 1. Plot Confusion Matrix
fig, ax = plt.subplots(1, 2, figsize=(18, 7)) # Create a figure with two subplots

ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize='true', ax=ax[0])
ax[0].set_title("Normalized Confusion Matrix")

RocCurveDisplay.from_estimator(xgb, X_test, y_test, ax=ax[1])
ax[1].set_title("ROC Curve")

plt.show()

## Conclusions

The results were satisfactory, taking into account that `recall` was the main metric for optimization. Test and training results are close, which is a good sign, and indicates the parameters have been decently tuned.

There's still a lot of room for imporivement. We can try different balancing techniques, feature engineering, feature selection, etc.

This is a very complex and interesting project, where the data analysis and machine learning can be really helpful.




